In [21]:
import json
from collections import defaultdict
import random
import string
import numpy as np
import operator
import math
import re
import gzip
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
import pandas as pd
import csv

In [2]:
# I use gz file so next cell
data =[]

with open('renttherunway_final_data.json', 'r') as file:
    for line in file:
        line_data = json.loads(line)
        data.append(line_data)


In [201]:

def readGz(path):
    for l in gzip.open(path, 'rt', encoding='utf-8'):
        if 'null' not in l:
            yield eval(l)
data = list(readGz("renttherunway_final_data.json.gz"))
for index, item in enumerate(data, start=1):
    item['unique_id'] = index
df = pd.read_csv('rent_sentiment.csv')



In [202]:
df['unique_id'] = range(1, 192463)

In [206]:
# I use this cell to drop na
data = pd.DataFrame(data)
merged_df = pd.merge(data, df[['unique_id','nltk_sentiment_compound_score','nltk_sentiment_compound_score_summary']], on='unique_id', how='left')
merged_df = merged_df.dropna()
clean_data = merged_df.to_dict(orient='records')


In [204]:
# drop rows that have incomplete columns 
clean_data = []
for d in merged_df:
    if len(d) == 19:
        clean_data.append(d)

In [207]:
print(len(clean_data))
print(clean_data[0])

146381
{'fit': 'fit', 'user_id': '420272', 'bust size': '34d', 'item_id': '2260466', 'weight': '137lbs', 'rating': '10', 'rented for': 'vacation', 'review_text': "An adorable romper! Belt and zipper were a little hard to navigate in a full day of wear/bathroom use, but that's to be expected. Wish it had pockets, but other than that-- absolutely perfect! I got a million compliments.", 'body type': 'hourglass', 'review_summary': 'So many compliments!', 'category': 'romper', 'height': '5\' 8"', 'size': 14, 'age': '28', 'review_date': 'April 20, 2016', 'unique_id': 1, 'nltk_sentiment_compound_score': 0.9453, 'nltk_sentiment_compound_score_summary': 0.5057}


In [142]:
# train, val, test split
random.seed(7)
random.shuffle(clean_data)

train_data = clean_data[:int(0.8*len(clean_data))]
valid_data = clean_data[int(0.8*len(clean_data)):int(0.9*len(clean_data))]
test_data = clean_data[int(0.9*len(clean_data)):]

In [189]:
def feature(d):
    f = [1]  #offset

    size = d['size']
    f.append(size)
    
    height_transform = re.findall(r'\d+', d['height'])
    height = int(height_transform[0]) * 12 + int(height_transform[1])
    f.append(height)

    weight = int(d['weight'][:-3])
    f.append(weight)
    
    bust = int(d['bust size'][:2])
    f.append(bust)
    cup = ord(d['bust size'][2])
    f.append(cup)

    age = int(d['age'])
    f.append(age)
    
    
    
    # one-hot encoding for rented for and body type
    rented_for_dict = {'date':0, 'everyday':1, 'formal affair':2, 'other':3, 'party':4, 'party: cocktail':5, 'vacation':6, 'wedding':7, 'work':8}
    f.append(1.0 * (d['rented for'] == 'date'))
    f.append(1.0 * (d['rented for'] == 'everyday'))
    f.append(1.0 * (d['rented for'] == 'formal affair'))
    f.append(1.0 * (d['rented for'] == 'other'))
    f.append(1.0 * (d['rented for'] == 'party'))
    f.append(1.0 * (d['rented for'] == 'party: cocktail'))
    f.append(1.0 * (d['rented for'] == 'vacation'))
    f.append(1.0 * (d['rented for'] == 'wedding'))
    f.append(1.0 * (d['rented for'] == 'work'))
    
    body_type_dict = {'apple':0, 'athletic':1, 'full bust':2, 'hourglass':3, 'pear':4, 'petite':5, 'straight & narrow':6}
    f.append(1.0 * (d["body type"] == 'apple'))
    f.append(1.0 * (d["body type"] == 'athletic'))
    f.append(1.0 * (d["body type"] == 'full bust'))
    f.append(1.0 * (d["body type"] == 'hourglass'))
    f.append(1.0 * (d["body type"] == 'pear'))
    f.append(1.0 * (d["body type"] == 'petite'))
    f.append(1.0 * (d["body type"] == 'straight & narrow'))
    

    ### add more features?
    ### processed review_text prediction results?
    ### e.g. f.append(sentiment_model.predict(d["review_text"]))

    f.append(d["nltk_sentiment_compound_score"])
    f.append(d["nltk_sentiment_compound_score_summary"])
    # label: change 3 categories to 
    if d['fit'] == 'fit':
        label = 1
    else:
        label = 0      # not fit

    return f, label

    

In [190]:
train_X = []
train_Y = []
for d in train_data:
    x, y = feature(d)
    train_X.append(x)
    train_Y.append(y)

valid_X = []
valid_Y = []
for d in valid_data:
    x, y = feature(d)
    valid_X.append(x)
    valid_Y.append(y)


test_X = []
test_Y = []
for d in test_data:
    x, y = feature(d)
    test_X.append(x)
    test_Y.append(y)


In [191]:
def calculate_accuracy(y, pred):
    return sum([a==b for a, b in zip(y, pred)]) / len(y)

In [192]:
### logistic regression

In [193]:
lr_mod = LogisticRegression(C=1)
lr_mod.fit(train_X, train_Y)



/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(C=1)

In [194]:
lr_predictions = lr_mod.predict(test_X)
lr_accuracy = calculate_accuracy(test_Y, lr_predictions)


In [195]:
lr_accuracy

0.739736320786939

In [ ]:
# no sentiment 0.7391215246943097

In [ ]:
# review summary sentiment 0.7390532140173509

In [ ]:
# review text sentiment 0.7392581460482274

In [ ]:
# both sentiment 0.739736320786939

In [150]:
lr_mod.coef_[0]

array([-1.08897630e-04, -2.81721855e-02,  1.85652117e-03,  6.61892837e-03,
        2.28294826e-02, -7.29617817e-03,  5.92762115e-03, -6.32160506e-02,
       -1.99084231e-01,  2.27176142e-01,  1.00888650e-02,  9.66611043e-03,
        2.04509383e-04, -2.91285678e-02,  9.32251241e-02, -4.90407995e-02,
       -1.40740607e-02,  2.99761013e-02, -5.83388322e-02,  2.98172045e-02,
       -5.99721142e-02,  4.59409401e-02,  2.65418636e-02])

In [15]:
### random forest

In [196]:
rf_model = RandomForestClassifier()
rf_model.fit(train_X, train_Y)

RandomForestClassifier()

In [197]:
rf_predictions = rf_model.predict(test_X)
rf_accuracy = calculate_accuracy(test_Y, rf_predictions)

In [198]:
rf_accuracy

0.7331784957988934

In [ ]:
# no sentiment 0.7024386911674295

In [ ]:
# sentiment review_text 0.7247762825329599

In [ ]:
# sentiment review_summary 0.7240248650864131

In [ ]:
# both sentiment 0.7331784957988934

In [ ]:
### use grid search to improve random forest classifier

In [ ]:
param_grid = {
    'n_estimators': [50, 100, 200, 300, 500, 800, 1600],
    'random_state': [6, 42, 100, 277, 832, 2022]
}

# Create the GridSearchCV object
grid_search = GridSearchCV(estimator=rf_model, param_grid=param_grid, cv=5, scoring='accuracy')

# Perform the grid search on your training data
grid_search.fit(train_X, train_Y)

best_params = grid_search.best_params_
print("Best Parameters:", best_params)